Including libraries and scripts

In [13]:
%matplotlib inline
import nbimport
import numpy as np
import scipy.sparse as spa
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.sparse.linalg as spla
import numpy.linalg as npl
from pyamg import smoothed_aggregation_solver, rootnode_solver
np.set_printoptions(linewidth=130)

In [14]:
from fsmfuncs import *
from ibmfuncs import *
from gridfuncs import *

Numerical grid for the fluid

In [15]:
x = np.linspace(-5,5,250)
y = np.linspace(-50,50,2500)

n, m = len(y)-1, len(x)-1

dy, dx = np.diff(y), np.diff(x)
dxmin = min(np.min(dx), np.min(dy)) 

# Pressure
yp, xp = 0.5*(y[1:] + y[:-1]), 0.5*(x[1:] + x[:-1]) 
dyp, dxp = np.diff(yp), np.diff(xp)
p = np.zeros( (n, m) )

# Horizontal velocity
yu, xu = yp, x[1:-1]
u = np.zeros( (n, m-1) )

# Vertical velocity
yv, xv = y[1:-1], xp
v = np.zeros( (n-1, m) )

Immersed boundary

In [16]:
r_ = 0.5
l = int((2*np.pi*r_)/dxmin) 
ang_ = 2*np.pi*np.arange(l)/l 

# Coordinates
xi = 0 + r_*np.cos(ang_)
eta = 40 + r_*np.sin(ang_)

ds = 2*np.pi*r_/l*np.ones(l) # r/l équivalent de sin teta donc de l'angle teta, ds est le pas sur la frontière

# Velocity
UB = np.zeros(2)
UBm1 = np.zeros(2)

# Number of lagrangian points
nB= len(xi)


Plot grid and immersed boundary

In [ ]:
plt.figure(figsize=(8,8))
X, Y = np.meshgrid(x, y) 
plt.plot(X, Y, 'b-');
plt.plot(X.T, Y.T, 'b-'); 
plt.plot(xi, eta, 'ro-'); 
plt.axis('equal')

Build matrices (I)

In [19]:
G, DuW, DuE, DvS, DvN = gradient(dxp, dyp)
R, iR = weight (dx, dy)
Mh, iMh = mass_hat (dxp, dyp)
Lh, Lux0, Lux1, Luy0, Luy1, Lvx0, Lvx1, Lvy0, Lvy1 = laplacian_hat(dx, dy, dxp, dyp)

L = Mh.dot(Lh.dot(iR))

M = Mh.dot(iR)
iM = R.dot(iMh)

iML = iM.dot(L)

In [ ]:
# Reynolds number
iRe = 1/200.0

# Time step
dt = 0.4* min(dxmin**2/iRe, dxmin) 

print(dt, dxmin**2/iRe, dxmin)

Build matrices (II)

In [21]:
A = (M/dt - 0.5*iRe*L).tocsr() 
B = (M/dt + 0.5*iRe*L).tocsr()

BN = dt*iM + (0.5*iRe)*dt**2*iML.dot(iM) + (0.5*iRe)**2*dt**3*iML.dot(iML.dot(iM))

In [22]:
# Velocity flux
q = R.dot(np.concatenate([u.ravel(), v.ravel()])) 
qast = q.copy()

UBast = UB.copy()

Num1, Nvm1 = advection_hat(dx, dy, dxp, dyp, iR.dot(q),  uS, uN, uW, uE, vS, vN, vW, vE)
Nu, Nv = Num1, Nvm1

GTBNG = G.T.dot(BN.dot(G))

Dimensionless parameters

In [ ]:
# Density ratio
rho = 2 
# Gravity
g = 0.5
# Volume
Vs = np.pi/4

Build matrices (III)

In [ ]:
IB = (rho/dt)*Vs*np.eye(2,2)
iIB = dt/(Vs*rho)*np.eye(2,2)
NB = - np.ones((2,2*nB))
NB[0,nB:]=np.zeros(nB)
NB[1,:nB]=np.zeros(nB)
    
GB=np.zeros(2)
GB[1]= Vs*(1-rho)*g


Time loop

In [ ]:
nt = int(50/dt)
print("Performing", nt, "steps")

residuals = np.zeros(nt)

# Drag and lift coefficients
CFx = np.zeros(nt)
CFy = np.zeros(nt)

# Vertical velocity of the cylinder
Vc = np.zeros(nt)

# Vertical position of the cylinder
Yc = np.zeros(nt)

for k in range(nt):   
    
    # Computing the regularization and the interpolation operators
    Eh = interpolation_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)
    Hh = regularization_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)

    E = Eh.dot(iR)
    H = Mh.dot(Hh)
    
    dQ = (Vs/dt)*(UB-UBm1)

    UBast[0]= UB[0]+dt/(rho*Vs)*(dQ[0]+GB[0])
    UBast[1]= UB[1]+dt/(rho*Vs)*(dQ[1]+GB[1])
    
    ru = iRe*(Lux0.dot(uW) + Lux1.dot(uE) + Luy0.dot(uS) + Luy1.dot(uN)) - 1.5*Nu + 0.5*Num1
    rv = iRe*(Lvx0.dot(vW) + Lvx1.dot(vE) + Lvy0.dot(vS) + Lvy1.dot(vN)) - 1.5*Nv + 0.5*Nvm1 - g
    
    bc1 = Mh.dot(np.concatenate([ru, rv]))
    r1 = B.dot(q.ravel()) + bc1
   
    # Solving the first linear equation
    mls = smoothed_aggregation_solver(A)
    accelerated_residuals = []
    qast= mls.solve( r1, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    bc2 = - (DuW.dot(uW*dy) + DuE.dot(uE*dy) + DvS.dot(vS*dx) + DvN.dot(vN*dx))
   
    GTBNET = G.T.dot(BN.dot(E.T))
    EBNG = E.dot(BN.dot(G))
    X = E.dot(BN.dot(E.T)) + NB.T.dot(iIB.dot(NB))
    
    e = spa.hstack((GTBNG,GTBNET))
    j = spa.hstack((EBNG,X))
    Q = spa.vstack((e,j)).tocsr()
    
    a = np.concatenate((G.T.dot(qast.ravel())+bc2, E.dot(qast.ravel())+ NB.T.dot(UBast)))
    
    # Solving the second linear equation
    mls1 = smoothed_aggregation_solver(Q)
    accelerated_residuals = []
    λ = mls1.solve(a, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    # Projection step
    qp1 = qast - BN.dot(G.dot(λ[:n*m])) - BN.dot(E.T.dot(λ[n*m:]))
    UB1 = UBast - iIB.dot(NB.dot(λ[n*m:]))
    
    residuals[k] = la.norm(qp1-q)/(la.norm(dt*qp1)) 
    
    # Forcing term
    F = iMh.dot(E.T.dot(λ[n*m:]))
    Fx, Fy = F[:n*(m-1)].reshape((n, m-1)), F[n*(m-1):].reshape((n-1,m))
    CFx[k] = 2*np.sum(Fx*dxp[np.newaxis,:]*dy[:,np.newaxis]) 
    CFy[k] = 2*np.sum(Fy*dx[np.newaxis,:]*dyp[:,np.newaxis])  
    
    #Updating coordinates
    eta = eta + UB[1]*dt    
    
    Yc[k]= eta[0]
    Vc[k]= UB[1]
    
    if k%100==0:
        print(k, k*dt, residuals[k], CFy[k], CFx[k], eta[0], UB[1])
   
    
    q = qp1.copy()
    UBm1 = UB.copy()
    UB = UB1.copy()
    
    Num1, Nvm1 = Nu, Nv
    Nu, Nv = advection_hat(dx, dy, dxp, dyp, iR.dot(q), uS, uN, uW, uE, vS, vN, vW, vE)
   

Velocity, pressure, vorticity and forcing term

In [ ]:
iRq = iR.dot(q)
u, v = iRq[:n*(m-1)].reshape((n, m-1)), iRq[n*(m-1):].reshape((n-1, m))
p = λ[:n*m].reshape((n,m))
f = λ[n*m:]
w = np.diff(v, axis=1)/dxp[np.newaxis,:]-np.diff(u,axis=0)/dyp[:,np.newaxis]

Display solution

In [ ]:
# Ploting velocity
x0, x1 = -5, 5
y0, y1 = 15, 35
plt.figure(figsize=(5.5*3,4))
plt.subplot(1,3,1) 
plt.pcolormesh(xu, yu, u, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

plt.subplot(1,3,2)
plt.pcolormesh(xv, yv, v, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

# Ploting pressure
plt.subplot(1,3,3)
plt.pcolormesh(xp, yp, p, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting vorticity
plt.figure(figsize=(5.5,4))
plt.pcolormesh(xu, yv, w, shading='gouraud', vmin= -5, vmax=5)
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting drag and lift coefficients
plt.plot(np.arange(len(CFx))*dt, CFx, 'b-',label='Cd')
plt.plot(np.arange(len(CFy))*dt, CFy, 'r-',label='Cl')
plt.xlim(0, 50)
plt.ylim(0, 9)
plt.xlabel('t')
plt.ylabel('Aerodynamic coefficients')

In [ ]:
# Ploting the vertical velocity of the cylinder
plt.plot(np.arange(len(CFy))*dt, Vc, 'r-')
plt.xlim(0, 50)
plt.ylim(0.5, 0.9)
plt.xlabel('t')
plt.ylabel('Vc')

In [ ]:
# Ploting the vertical position of the cylinder
plt.plot(np.arange(len(CFy))*dt, Yc, 'b-')
plt.xlim(0, 30)
plt.ylim(10, 45)
plt.xlabel('t')
plt.ylabel('Yc')